<a href="https://colab.research.google.com/github/Sundaypaul/PROJECTS/blob/main/DISEASE_Diagnosis_Prediction_Model_With_RandomForestClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The dataset used in this analysis was sourced from kaggle with the title [Disease-Symptom Dataset](https://www.kaggle.com/datasets/dhivyeshrk/diseases-and-symptoms-dataset) and it was uploaded by DHIVYESH R. K.

Here is the Description of the dataset;
The dataset contains disease names along with the symptoms faced by the respective patient. There are a total of 773 unique diseases and 377 symptoms, with ~246,000 rows. The dataset was artificially generated, preserving Symptom Severity and Disease Occurrence Possibility.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from re import X
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import plotly.figure_factory as ff
import numpy as np
from sklearn.metrics import confusion_matrix
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, roc_auc_score
from json import encoder
import category_encoders as ce
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
pd.options.display.max_columns = 10
pd.options.display.float_format = '{:.2f}'.fo

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
file_path = '/content/drive/MyDrive/Dataset/Disease_symptom_and_patient_profile_dataset.csv'
Disease_symptoms = pd.read_csv(file_path)
df = Disease_symptoms

#view the first five rows of the dataset
print('Dataset loaded successfully.')
print(f'Total number of rows: {len(df)}\n\n')
df.head()

Dataset loaded successfully.
Total number of rows: 349




,Disease,Fever,Cough,Fatigue,Difficulty Breathing,Age,Gender,Blood Pressure,Cholesterol Level,Outcome Variable
0,Influenza,Yes,No,Yes,Yes,19,Female,Low,Normal,Positive
1,Common Cold,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
2,Eczema,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
3,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
4,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive


In [31]:
df

,Disease,Fever,Cough,Fatigue,Difficulty Breathing,Age,Gender,Blood Pressure,Cholesterol Level,Outcome Variable
0,Influenza,Yes,No,Yes,Yes,19,Female,Low,Normal,Positive
1,Common Cold,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
2,Eczema,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
3,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
5,Eczema,Yes,No,No,No,25,Female,Normal,Normal,Positive
...,...,...,...,...,...,...,...,...,...,...
341,Tonsillitis,Yes,Yes,Yes,No,70,Female,High,High,Positive
342,Williams Syndrome,No,No,Yes,No,70,Female,Normal,Normal,Positive
343,Stroke,Yes,No,Yes,No,80,Female,High,High,Positive
345,Stroke,Yes,No,Yes,No,85,Male,High,High,Positive


#Dataset Overview

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 0 to 347
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Disease               300 non-null    object
 1   Fever                 300 non-null    object
 2   Cough                 300 non-null    object
 3   Fatigue               300 non-null    object
 4   Difficulty Breathing  300 non-null    object
 5   Age                   300 non-null    int64 
 6   Gender                300 non-null    object
 7   Blood Pressure        300 non-null    object
 8   Cholesterol Level     300 non-null    object
 9   Outcome Variable      300 non-null    object
dtypes: int64(1), object(9)
memory usage: 25.8+ KB


In [33]:
df.isna().sum().any()

False

The result indicate that the dataset contain 1 numerical column (Age) and does not have any missing values.

In [34]:
df

,Disease,Fever,Cough,Fatigue,Difficulty Breathing,Age,Gender,Blood Pressure,Cholesterol Level,Outcome Variable
0,Influenza,Yes,No,Yes,Yes,19,Female,Low,Normal,Positive
1,Common Cold,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
2,Eczema,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
3,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
5,Eczema,Yes,No,No,No,25,Female,Normal,Normal,Positive
...,...,...,...,...,...,...,...,...,...,...
341,Tonsillitis,Yes,Yes,Yes,No,70,Female,High,High,Positive
342,Williams Syndrome,No,No,Yes,No,70,Female,Normal,Normal,Positive
343,Stroke,Yes,No,Yes,No,80,Female,High,High,Positive
345,Stroke,Yes,No,Yes,No,85,Male,High,High,Positive


**Check out for duplicates**

In [35]:
duplicate_rows = df.duplicated().sum()
print(f'Number of duplicate rows: {duplicate_rows}')

Number of duplicate rows: 0


In [36]:
df = df.drop_duplicates()
df.drop_duplicates(inplace=True)

In [37]:
cleaned_df = df.copy()

In [38]:
duplicate_rows = cleaned_df.duplicated().sum()
print(f'Number of duplicate rows: {duplicate_rows}')

Number of duplicate rows: 0


Duplicates has been successfully removed

In [39]:
#To check the statistic of the numerical column
cleaned_df.describe()

,Age
count,300.00
mean,45.76
std,12.60
min,19.00
25%,35.00
50%,45.00
75%,55.00
max,90.00


In [40]:
categorical_var = cleaned_df.select_dtypes(include=['object']).columns
for column in categorical_var:
  print(f'{column}: {cleaned_df[column].value_counts()}')

Disease: Disease
Asthma                            16
Osteoporosis                      12
Stroke                            11
Hypertension                      10
Migraine                          10
                                  ..
Fibromyalgia                       1
Eating Disorders (Anorexia,...     1
Chickenpox                         1
Rabies                             1
Williams Syndrome                  1
Name: count, Length: 116, dtype: int64
Fever: Fever
No     157
Yes    143
Name: count, dtype: int64
Cough: Cough
No     160
Yes    140
Name: count, dtype: int64
Fatigue: Fatigue
Yes    212
No      88
Name: count, dtype: int64
Difficulty Breathing: Difficulty Breathing
No     227
Yes     73
Name: count, dtype: int64
Gender: Gender
Female    152
Male      148
Name: count, dtype: int64
Blood Pressure: Blood Pressure
High      143
Normal    139
Low        18
Name: count, dtype: int64
Cholesterol Level: Cholesterol Level
High      140
Normal    130
Low        30
Name: count,

## **Exploratory data analysis**

Analyze the label(Outcome variable)

In [41]:
fig = px.histogram(cleaned_df, x='Outcome Variable', title='Distribution of Outcome', color= 'Outcome Variable', template= 'plotly_dark')

fig.show()

In [42]:
categorical_var
for column in categorical_var:
  fig = px.histogram(cleaned_df, x=column, title=f'Distribution of {column}', color=column, template= 'plotly_dark')
  fig.show()

In [43]:
#for the numerical data (Age)
fig= px.histogram(cleaned_df, x='Age', title='Distribution of Age', color= 'Age', template= 'plotly_dark', nbins = 10)
fig.show()

**Visualize label relationship with features**

In [44]:
for column in categorical_var:
  fig = px.histogram(cleaned_df, x=column, color='Outcome Variable', title=f'{column} vs Outcome Variable', template= 'plotly_dark', barmode= 'group')
  fig.show()

In [45]:
#for the numerical data (Age)
fig= px.histogram(cleaned_df, x='Outcome Variable', title='Distribution of Age vs Outcome variable', color= 'Outcome Variable', y= "Age", template= 'plotly_dark', nbins = 10)
fig.show()

In [200]:
#for the numerical data (Age)
fig= px.scatter(cleaned_df, y='Outcome Variable', title='Distribution of Age vs Outcome variable', color= 'Outcome Variable', x= "Age", template= 'plotly_dark')
fig.show()

**Feature Engineering/Data preprocessing**

In [47]:
categorical_var

Index(['Disease', 'Fever', 'Cough', 'Fatigue', 'Difficulty Breathing',
       'Gender', 'Blood Pressure', 'Cholesterol Level', 'Outcome Variable'],
      dtype='object')

**Features encoding**

In [48]:
map_bp = {'High': 3, 'Normal': 2, 'Low': 1}
cleaned_df['Blood Pressure_encoded'] = cleaned_df['Blood Pressure'].map(map_bp)

map_chol = {'High': 3, 'Normal': 2, 'Low': 1}
cleaned_df['Cholesterol_encoded'] = cleaned_df['Cholesterol Level'].map(map_chol)

map_fever = {'Yes': 1, 'No': 0}
cleaned_df['Fever_encoded'] = cleaned_df['Fever'].map(map_fever)

map_Outcome_Variable = {'Positive': 1, 'Negative': 0}
cleaned_df['Outcome Variable_encoded'] = cleaned_df['Outcome Variable'].map(map_Outcome_Variable)

feature_map = ['Cough', 'Fatigue', 'Difficulty Breathing']
for feature in feature_map:
  map_feature = {'Yes': 1, 'No': 0}
  cleaned_df[f'{feature}_encoded'] = cleaned_df[feature].map(map_feature)

In [49]:
cleaned_df.head()

,Disease,Fever,Cough,Fatigue,Difficulty Breathing,...,Fever_encoded,Outcome Variable_encoded,Cough_encoded,Fatigue_encoded,Difficulty Breathing_encoded
0,Influenza,Yes,No,Yes,Yes,...,1,1,0,1,1
1,Common Cold,No,Yes,Yes,No,...,0,0,1,1,0
2,Eczema,No,Yes,Yes,No,...,0,0,1,1,0
3,Asthma,Yes,Yes,No,Yes,...,1,1,1,0,1
5,Eczema,Yes,No,No,No,...,1,1,0,0,0


In [50]:
cleaned_df[['Disease']]

,Disease
0,Influenza
1,Common Cold
2,Eczema
3,Asthma
5,Eczema
...,...
341,Tonsillitis
342,Williams Syndrome
343,Stroke
345,Stroke


**Encode the disease column**

In [51]:
cleaned_df[['Disease']]

,Disease
0,Influenza
1,Common Cold
2,Eczema
3,Asthma
5,Eczema
...,...
341,Tonsillitis
342,Williams Syndrome
343,Stroke
345,Stroke


In [52]:
disease_mapping = {label: disease for label, disease in enumerate(cleaned_df['Disease'].unique())}
print(disease_mapping)

{0: 'Influenza', 1: 'Common Cold', 2: 'Eczema', 3: 'Asthma', 4: 'Hyperthyroidism', 5: 'Allergic Rhinitis', 6: 'Anxiety Disorders', 7: 'Diabetes', 8: 'Gastroenteritis', 9: 'Pancreatitis', 10: 'Rheumatoid Arthritis', 11: 'Depression', 12: 'Liver Cancer', 13: 'Stroke', 14: 'Urinary Tract Infection', 15: 'Dengue Fever', 16: 'Hepatitis', 17: 'Kidney Cancer', 18: 'Migraine', 19: 'Muscular Dystrophy', 20: 'Sinusitis', 21: 'Ulcerative Colitis', 22: 'Bipolar Disorder', 23: 'Bronchitis', 24: 'Cerebral Palsy', 25: 'Colorectal Cancer', 26: 'Hypertensive Heart Disease', 27: 'Multiple Sclerosis', 28: 'Myocardial Infarction (Heart...', 29: 'Urinary Tract Infection (UTI)', 30: 'Osteoporosis', 31: 'Pneumonia', 32: 'Atherosclerosis', 33: 'Chronic Obstructive Pulmonary...', 34: 'Epilepsy', 35: 'Hypertension', 36: 'Obsessive-Compulsive Disorde...', 37: 'Psoriasis', 38: 'Rubella', 39: 'Cirrhosis', 40: 'Conjunctivitis (Pink Eye)', 41: 'Liver Disease', 42: 'Malaria', 43: 'Spina Bifida', 44: 'Kidney Disease',

In [53]:
!pip install category-encoders


In [54]:
encoder = ce.BinaryEncoder(cols=['Disease'])
cleaned_df = encoder.fit_transform(cleaned_df)

In [55]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 0 to 347
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Disease_0                     300 non-null    int64 
 1   Disease_1                     300 non-null    int64 
 2   Disease_2                     300 non-null    int64 
 3   Disease_3                     300 non-null    int64 
 4   Disease_4                     300 non-null    int64 
 5   Disease_5                     300 non-null    int64 
 6   Disease_6                     300 non-null    int64 
 7   Fever                         300 non-null    object
 8   Cough                         300 non-null    object
 9   Fatigue                       300 non-null    object
 10  Difficulty Breathing          300 non-null    object
 11  Age                           300 non-null    int64 
 12  Gender                        300 non-null    object
 13  Blood Pressure           

In [28]:
cleaned_df.head()

,Disease_0,Disease_1,Disease_2,Disease_3,Disease_4,...,Fever_encoded,Outcome Variable_encoded,Cough_encoded,Fatigue_encoded,Difficulty Breathing_encoded
0,0,0,0,0,0,...,1,1,0,1,1
1,0,0,0,0,0,...,0,0,1,1,0
2,0,0,0,0,0,...,0,0,1,1,0
3,0,0,0,0,1,...,1,1,1,0,1
5,0,0,0,0,0,...,1,1,0,0,0


In [57]:
cleaned_df[['Disease_0', 'Disease_1','Disease_2', 'Disease_3', 'Disease_4', 'Disease_5', 'Disease_6']]

,Disease_0,Disease_1,Disease_2,Disease_3,Disease_4,Disease_5,Disease_6
0,0,0,0,0,0,0,1
1,0,0,0,0,0,1,0
2,0,0,0,0,0,1,1
3,0,0,0,0,1,0,0
5,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...
341,1,1,1,0,0,1,1
342,1,1,1,0,1,0,0
343,0,0,0,1,1,1,0
345,0,0,0,1,1,1,0


In [80]:
#To check out the labeled as coded
encoded_label = 5
disease_name = disease_mapping[encoded_label]
disease_name

'Allergic Rhinitis'

**Normalize age column**

In [88]:
normalized_age = (cleaned_df['Age'] - cleaned_df['Age'].mean()) / cleaned_df['Age'].std()
cleaned_df['Age_normalized'] = normalized_age


In [89]:
cleaned_df.head()

,Disease_0,Disease_1,Disease_2,Disease_3,Disease_4,...,Outcome Variable_encoded,Cough_encoded,Fatigue_encoded,Difficulty Breathing_encoded,Age_normalized
0,0,0,0,0,0,...,1,0,1,1,-2.12
1,0,0,0,0,0,...,0,1,1,0,-1.65
2,0,0,0,0,0,...,0,1,1,0,-1.65
3,0,0,0,0,1,...,1,1,0,1,-1.65
5,0,0,0,0,0,...,1,0,0,0,-1.65


**Split data**

In [101]:
X = cleaned_df.drop(columns=['Outcome Variable', 'Outcome Variable_encoded',  'Fever', 'Cough', 'Fatigue', 'Difficulty Breathing', 'Age', 'Gender', 'Blood Pressure', 'Cholesterol Level'])
y = cleaned_df['Outcome Variable_encoded']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [104]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(240, 14)
(30, 14)
(30, 14)


**Train model**

In [151]:
#import model for classification
rf_model =RandomForestClassifier(random_state=42)

In [152]:
#train model
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

**Validate model**

In [153]:
#Validate with X_val
y_val_pred = rf_model.predict(X_val)

In [154]:
#To know the accuracy of the validity
validity_accuracy = accuracy_score(y_val, y_val_pred)

In [155]:
print(f'The validity accuracy of the model is {validity_accuracy:.2f}')

The validity accuracy of the model is 0.73


**Test model**

In [156]:
y_test_pred = rf_model.predict(X_test)

In [157]:
Test_accuracy = accuracy_score(y_test, y_test_pred)
print(f'The test accuracy of the model is {validity_score:.2f}')

The test accuracy of the model is 0.73


**Fine tune model**

In [177]:
param_grid = {'n_estimators': [200, 300, 500], 'max_depth': [12, 13, 15], 'min_samples_split': [12,13,14]}

grid_search = GridSearchCV(RandomForestClassifier(random_state = 42), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [12, 13, 15],
                         'min_samples_split': [12, 13, 14],
                         'n_estimators': [200, 300, 500]},
             scoring='accuracy')

In [178]:
Best_params = grid_search.best_params_

In [179]:
Best_params

{'max_depth': 12, 'min_samples_split': 12, 'n_estimators': 500}

In [180]:
Best_rf_model = grid_search.best_estimator_

In [181]:

Best_rf_model

RandomForestClassifier(max_depth=12, min_samples_split=12, n_estimators=500,
                       random_state=42)

**Analyse Performance**

In [182]:

classification_report = classification_report(y_test, y_test_pred)
confusion_mat = confusion_matrix(y_test, y_test_pred)

print(f'The classification report for the test set is: \n {classification_report}\n\n')
print(f'The confusion matrix for the test set is: \n{confusion_mat}')

The classification report for the test set is: 
               precision    recall  f1-score   support

           0       0.73      0.53      0.62        15
           1       0.63      0.80      0.71        15

    accuracy                           0.67        30
   macro avg       0.68      0.67      0.66        30
weighted avg       0.68      0.67      0.66        30



The confusion matrix for the test set is: 
[[ 8  7]
 [ 3 12]]


In [187]:


# Extract feature importance from the best model
feature_importances = Best_rf_model.feature_importances_
features = X_train.columns

# Create a DataFrame for feature importances
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Plot feature importance with Plotly Express
fig = px.bar(
    importance_df,
    x='Importance',
    y='Feature',
    orientation='h',  # Horizontal bar chart
    title='Feature Importance',
    color='Importance',
    color_continuous_scale='Blues',
    template='plotly_dark'  # Optional for dark theme
)

fig.update_layout(
    xaxis_title='Importance',
    yaxis_title='Features',
    yaxis=dict(autorange="reversed")  # Reverse the order of features on the y-axis
)

fig.show()


In [184]:
df

,Disease,Fever,Cough,Fatigue,Difficulty Breathing,Age,Gender,Blood Pressure,Cholesterol Level,Outcome Variable
0,Influenza,Yes,No,Yes,Yes,19,Female,Low,Normal,Positive
1,Common Cold,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
2,Eczema,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
3,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
5,Eczema,Yes,No,No,No,25,Female,Normal,Normal,Positive
...,...,...,...,...,...,...,...,...,...,...
341,Tonsillitis,Yes,Yes,Yes,No,70,Female,High,High,Positive
342,Williams Syndrome,No,No,Yes,No,70,Female,Normal,Normal,Positive
343,Stroke,Yes,No,Yes,No,80,Female,High,High,Positive
345,Stroke,Yes,No,Yes,No,85,Male,High,High,Positive


In [189]:


# Predict on test set
y_test_pred = Best_rf_model.predict(X_test)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_test_pred)

# Create annotations for heatmap
labels = np.array([['True Negative', 'False Positive'], ['False Negative', 'True Positive']])
annotations = [[f"{val}\n({label})" for val, label in zip(row, labels_row)] for row, labels_row in zip(conf_matrix, labels)]

# Create heatmap with Plotly
fig = ff.create_annotated_heatmap(
    z=conf_matrix,
    x=['Negative', 'Positive'],  # Predicted labels
    y=['Negative', 'Positive'],  # True labels
    annotation_text=annotations,
    colorscale='Blues',
    showscale=True
)

# Update layout for better display
fig.update_layout(
    title='Confusion Matrix Heatmap',
    xaxis_title='Predicted Label',
    yaxis_title='True Label',
    template='plotly_dark',  # Optional: Use dark theme
)

fig.show()


In [195]:


# Predict probabilities for the positive class
y_test_proba = Best_rf_model.predict_proba(X_test)[:, 1]

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, y_test_proba)
roc_auc = roc_auc_score(y_test, y_test_proba)

# Create ROC curve plot
fig = go.Figure()

# Add ROC curve
fig.add_trace(
    go.Scatter(
        x=fpr,
        y=tpr,
        mode='lines',
        name=f"ROC Curve (AUC = {roc_auc:.2f})",
        line=dict(color='blue', width=2)
    )
)

# Add diagonal line for random guess
fig.add_trace(
    go.Scatter(
        x=[0, 1],
        y=[0, 1],
        mode='lines',
        name='Random Guess',
        line=dict(color='gray', dash='dash')
    )
)

# Customize layout
fig.update_layout(
    title="Receiver Operating Characteristic (ROC) Curve",
    xaxis_title="False Positive Rate",
    yaxis_title="True Positive Rate",
    xaxis=dict(showgrid=True, zeroline=False),
    yaxis=dict(showgrid=True, zeroline=False),
    template="plotly_dark",  # Optional: Use dark theme
    legend=dict(x=0.8, y=0.2)
)

# Show the plot
fig.show()
